# Taxi problem: Q-Learning
 
A self-driving taxi can pick up passengers at one of a set of fixed locations, drop them off at another location, and get there in the quickest amount of time while avoiding obstacles.

**Credits:** Machine Learning, Data Science and Deep Learning with Python (Udemy)

In [1]:
import gym
import random

random.seed(1234)

streets = gym.make("Taxi-v2").env
streets.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



Legend:
-  R, G, B, and Y are pickup or dropoff locations.
-  The BLUE letter indicates where we need to pick someone up from.
-  The MAGENTA letter indicates where that passenger wants to go to.
-  The solid lines represent walls that the taxi cannot cross.
-  The filled rectangle represents the taxi itself - it's yellow when empty, and green when carrying a passenger.

The state of this world at any time can be defined by:
-  Where the taxi is (one of 5x5 = 25 locations)
-  What the current destination is (4 possibilities)
-  Where the passenger is (5 possibilities: at one of the destinations, or inside the taxi)

**Total:** 25 x 4 x 5 = 500 possible states that describe our world.

<u>Six possible actions:</u>

-  Move South (S), East (E), North (N), or West(W)
-  Pickup a passenger
-  Drop off a passenger


<u>Q-Learning rewards and penalties:</u>

-  A successfull drop-off yields +20 points
-  Every time step taken while driving a passenger yields a -1 point penalty
-  Picking up or dropping off at an illegal location yields a -10 point penalty
* **Note**: Moving across a wall just isn't allowed at all.

In [28]:
#define initial state: Taxi location (2,3), Passenger pickup location (2), Destination location (0)
initial_state = streets.encode(2, 3, 2, 0)

streets.s = initial_state

streets.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


#### Rewards:

In [12]:
translation={0:"South",1:"North",2:"East",3:"West",4:"PickUp",5:"DropOff"}

In [14]:
{translation[i]:streets.P[initial_state][i] for i in translation.keys()}

{'South': [(1.0, 368, -1, False)],
 'North': [(1.0, 168, -1, False)],
 'East': [(1.0, 288, -1, False)],
 'West': [(1.0, 248, -1, False)],
 'PickUp': [(1.0, 268, -10, False)],
 'DropOff': [(1.0, 268, -10, False)]}

In [13]:
streets.P[initial_state]

{0: [(1.0, 368, -1, False)],
 1: [(1.0, 168, -1, False)],
 2: [(1.0, 288, -1, False)],
 3: [(1.0, 248, -1, False)],
 4: [(1.0, 268, -10, False)],
 5: [(1.0, 268, -10, False)]}

 Format: 
<br>
 <center>{<b>Key</b>: [(<b>probability of that action</b>,<b>results at the next stage if action occurs</b>, <b>reward for that action</b>, <b>if a succesful dropoff took place</b>)]}</center>
<br>
<br>
So for example, moving North from this state would put us into state number 368, incur a penalty of -1 for taking up time, and does not result in a successful dropoff.

## Q-learning
First we need to train our model. At a high level, we'll train over 10,000 simulated taxi runs. For each run, we'll step through time, with a 10% chance at each step of making a random, exploratory step instead of using the learned Q values to guide our actions.

In [15]:
import numpy as np

q_table = np.zeros([streets.observation_space.n, streets.action_space.n])

#Hyperparameters:
learning_rate = 0.1
discount_factor = 0.6
exploration = 0.1
epochs = 10000

for taxi_run in range(epochs):
    state = streets.reset()
    done = False
    
    while not done:
        random_value = random.uniform(0, 1)
        if (random_value < exploration):
            action = streets.action_space.sample() # Explore a random action
        else:
            action = np.argmax(q_table[state]) # Use the action with the highest q-value
            
        next_state, reward, done, info = streets.step(action)
        
        prev_q = q_table[state, action]
        next_max_q = np.max(q_table[next_state])
        new_q = (1 - learning_rate) * prev_q + learning_rate * (reward + discount_factor * next_max_q)
        q_table[state, action] = new_q
        
        state = next_state
        
        

So now we have a table of Q-values that can be quickly used to determine the optimal next step for any given state! Let's check the table for our initial state above:

In [16]:
q_table[initial_state]

array([-2.4096594 , -2.41347164, -2.39555263, -2.3639511 , -8.65417899,
       -6.85881649])

The lowest q-value here corresponds to the action "go West", which makes sense - that's the most direct route toward our destination from that point. It seems to work! Let's see it in action!

In [17]:
from IPython.display import clear_output
from time import sleep

for tripnum in range(1, 11):
    state = streets.reset()
   
    done = False
    trip_length = 0
    
    while not done and trip_length < 25:
        action = np.argmax(q_table[state])
        next_state, reward, done, info = streets.step(action)
        clear_output(wait=True)
        print("Trip number " + str(tripnum) + " Step " + str(trip_length))
        print(streets.render(mode='ansi'))
        sleep(.5)
        state = next_state
        trip_length += 1
        
    sleep(2)
    

Trip number 10 Step 11
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

